<a href="https://colab.research.google.com/github/diegonogare/Workshop-AnalisePreditivaDeManutencao/blob/main/Workshop_AnalisePreditivaDeManutencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do ambiente

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn.ensemble import IsolationForest
import warnings

warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 7)

print("Bibliotecas importadas com sucesso!")

# Carregamento dos Dados

## Carregar o dataset


Carregar o dataset diretamente do [Github](https://github.com/diegonogare/Workshop-AnalisePreditivaDeManutencao/blob/main/conjuntoDados.csv).

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/diegonogare/Workshop-AnalisePreditivaDeManutencao/refs/heads/main/conjuntoDados.csv',
                 names=['ID', 'Timestamp', 'Temperatura', 'Sinais', 'Desligou'])

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Dia'] = df['Timestamp'].dt.day
df['DiaSemana'] = df['Timestamp'].dt.day_name()
df['Hora'] = df['Timestamp'].dt.time

df = df[df['Dia'] != (7)]
df = df[df['Dia'] != (29)]


## Conhecer o básico do dataset

Após o dataset ser carregado, é importante analisar seu conteúdo para conhecer o que há lá dentro.

In [ ]:
print("\nDataset carregado com sucesso!")
print("Formato do dataset (linhas x colunas):", df.shape)

print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações do dataset:")
df.info()

print("\nEstatística descritivas:")
print(df.describe())

print("\nValores nulos por Coluna:")
print(df.isnull().sum())

# Análise Exploratória Descritiva (EDA)

In [ ]:
colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
colunas_numericas

colunas_numericas.remove('ID') #ID da transação
colunas_numericas.remove('Desligou') #Flag se desligou ou não
colunas_numericas


In [ ]:
for col in colunas_numericas:
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribuição de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()
    print("\n")

In [ ]:
for col in colunas_numericas:
    sns.boxplot(x='Desligou', y=col, data=df)
    plt.title(f'{col} vs. Desligou')
    plt.show()
    print("\n")

In [ ]:
#Pela exploração que fizemos, apesar de Dia ser um valor numério, não tem relação com o objetivo que temos. Vamos remover
colunas_numericas.remove('Dia')

In [ ]:
correlation_matrix = df[colunas_numericas + ['Desligou']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

### 1 - Quais horários a temperatura da fábrica é mais alta?

In [ ]:
temperatura = df.copy()
temperatura.drop(columns=['Timestamp', 'Dia', 'Sinais', 'Desligou'], inplace=True )

temperatura.groupby('Hora')['Temperatura'].mean('Temperatura').plot(kind='line', marker='')
plt.title('Temperatura Média por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Temperatura Média (°C)')
plt.grid(True)
plt.show()


In [ ]:
for dia in df['Dia'].unique():
    temperaturaDiaria = df.copy()
    temperaturaDiaria = temperaturaDiaria[temperaturaDiaria['Dia'] == dia]
    temperaturaDiaria.drop(columns=['Timestamp', 'Sinais', 'Desligou'], inplace=True )
    diaSemana = temperaturaDiaria['DiaSemana'].iloc[0]

    temperaturaDiaria.groupby('Hora')['Temperatura'].mean('Temperatura').plot(kind='line', marker='')
    plt.title(f'Temperatura Média por Hora do {dia} - {diaSemana}')
    plt.xlabel('Hora do Dia')
    plt.ylabel('Temperatura Média (°C)')
    plt.grid(True)
    plt.show()

    print("\n")


### 2 - Quais temperaturas normalmente levam a máquina a parar de funcionar?


In [ ]:
maxFuncionando = df[df['Desligou'] == 0]['Temperatura'].max()
minNaoFuncionando = df[df['Desligou'] == 1]['Temperatura'].min()

sns.stripplot(x='Desligou', y='Temperatura', data=df, jitter=True, alpha=0.5)
plt.title('Temperatura vs. estado do equipamento (com Jitter)')
plt.xlabel('Máquina Parou (0 = Não, 1 = Sim)')
plt.ylabel('Temperatura (°C)')
plt.hlines(y=[maxFuncionando, minNaoFuncionando], xmin=-0.5, xmax=1.5, colors='red', linestyles='dashed')
plt.text(x=0.5, y=minNaoFuncionando - 1, s=minNaoFuncionando, color='red' )
plt.text(x=0.5, y=maxFuncionando + 0.5, s=maxFuncionando, color='red' )
plt.show()

In [ ]:
df['Sinais'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribuição da quantidade de Sinais coletados')
plt.xlabel('Quantidade de Sinais')
plt.ylabel('Frequência')
plt.show()

In [ ]:
sns.boxplot(x='Sinais', y='Temperatura', hue='Desligou', data=df)
plt.title('Temperatura vs. sinais por estado do equipamento')
plt.xlabel('Sinais')
plt.ylabel('Temperatura (°C)')
plt.legend(title='Desligou')
plt.show()


# Classificação com Regressão Logística

In [ ]:
X = df[colunas_numericas] # Features
y = df['Desligou'] # Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Tamanho do Treino: {X_train.shape}, Tamanho do Teste: {X_test.shape}")


In [ ]:
# Escalonamento (importante para muitos modelos)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
seed = 42
log_reg = LogisticRegression(random_state=seed)
log_reg.fit(X_train_scaled, y_train)
print("Modelo treinado!")

In [ ]:
print("\nRealizando previsões e avaliando o modelo...")
y_pred = log_reg.predict(X_test_scaled)
y_proba = log_reg.predict_proba(X_test_scaled) # Probabilidades para a classe 1 (Desligou)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do Modelo: {accuracy:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

print(classification_report(y_test, y_pred))

In [ ]:
# Coeficientes
print(f"\nCoeficiente (Temperatura): {log_reg.coef_}")
print(f"Intercepto: {log_reg.intercept_}")


In [ ]:
# Visualização da Curva Sigmoide (para uma feature)
X_test_sorted_indices = np.argsort(X_test['Temperatura'])
X_test_sorted = X_test['Temperatura'].iloc[X_test_sorted_indices]
y_proba_lr_sorted = y_proba[X_test_sorted_indices, 1]

y_test_sorted = y_test.iloc[X_test_sorted_indices]


In [ ]:
jitter = 0.1
y_jitter = y_test_sorted + np.random.uniform(-jitter, jitter, size=len(y_test_sorted))

plt.scatter(X_test_sorted, y_jitter, color='gray', zorder=20, alpha=0.05, label='Dados Reais (Teste)')

plt.plot(X_test_sorted, y_proba_lr_sorted, color='red', lw=3, label='Curva Sigmoide Predita')

plt.ylabel('Probabilidade de Desligar / Estado Real')
plt.xlabel('Temperatura (°C)')
plt.title('Regressão Logística: Probabilidade de Desligar vs. Temperatura')
plt.legend()
plt.show()

In [ ]:
def predizerQuebra(Temperatura):
  input_scaled = scaler.transform(Temperatura)

  predicted_class = log_reg.predict(input_scaled)
  prediction_proba = log_reg.predict_proba(input_scaled)

  predicted_class = log_reg.predict(input_scaled)
  prediction_proba = log_reg.predict_proba(input_scaled)
  prob_desligou = prediction_proba[0][1]

  print(f"\nTemperatura informada: {Temperatura[0][0]}°C")
  print(f"Classe Predita: {'Desligou' if predicted_class == 1 else 'Não Desligou'}")
  print(f"Probabilidade de Desligar: {prob_desligou:.4f}")

  plotarPredicao(Temperatura[0][0], prob_desligou)

In [ ]:
def plotarPredicao(Temperatura, prob_desligou):
  X_test_sorted_indices = np.argsort(X_test['Temperatura'])
  X_test_sorted = X_test['Temperatura'].iloc[X_test_sorted_indices]
  y_proba_lr_sorted = y_proba[X_test_sorted_indices, 1]

  y_test_sorted = y_test.iloc[X_test_sorted_indices]

  jitter = 0.1
  y_jitter = y_test_sorted + np.random.uniform(-jitter, jitter, size=len(y_test_sorted))
  plt.scatter(X_test_sorted, y_jitter, color='gray', zorder=20, alpha=0.05, label='Dados Reais (Teste)')
  plt.plot(X_test_sorted, y_proba_lr_sorted, color='red', lw=3, label='Curva Sigmoide Predita')

  # Plote o ponto da temperatura de input e sua probabilidade predita
  plt.scatter(Temperatura, prob_desligou, color='green', s=100, zorder=25, label=f'Input ({Temperatura}°C)')

  plt.ylabel('Probabilidade de Desligar / Estado Real')
  plt.xlabel('Temperatura (°C)')
  plt.title('Regressão Logística: Probabilidade de Desligar vs. Temperatura com Input')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
input_temperature = float(input("Por favor, digite a temperatura para previsão (ex: 25.5): "))

#input_temperature = float(25.0)
input_sinais = float(10)

novaTemperatura = np.array([[input_temperature, input_sinais]])

predizerQuebra(novaTemperatura)

# Detecção de Anomalias com Isolation Forest